# Popular posts on Hacker News

In this project, I am reviewing the Hacker News website.  This site is a popular forum that is similar to Reddit and used by those in technology and startups.  The dataset I am using has been reduced from the [original](https://www.kaggle.com/hacker-news/hacker-news-posts) 300,000 or so rows to approximately 20,000.  This reduction was done by first removing all posts that received no comments, then randomly selecting from the remaining records.  

The purpose of this project is to determine popularity of an article based on reader interaction.  That is why we are looking at commented posts.  Further we will examine two types of posts in particular, `Ask HN` and `Show HN`:  
- Ask HN: Users submit these posts to ask the Hacker News comunity a specific question.
    - Examples:
    - *Ask HN: How to improve my personal website?*
    - *Ask HN: Am I the only one outraged by Twitter shutting down share counts?*
    - *Ask HN: Aby recent changes to CSS that broke mobile?*
- Show HN: Users submit these posts to show the community a project, product or generally something interesting.
    - Examples:
    - *Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'*
    - *Show HN: Something pointless I made*
    - *Show HN: Shanhu.io, a programming playground powered by e8vm*

By examining these types of posts, we hope to answer two questions:
1. Do Ask HN or Show HN receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
hn_header = hn[0]
hn = hn[1:]
print('Column titles: ',hn_header,'\n')
print('Example of records: ',hn[:5],'\n')
print('Number of records: ',len(hn))

Column titles:  ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

Example of records:  [['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbel

As we can see by the column titles, the attributes are broken down by:
    
- `id`: The unique identifier from Hacker News for the post
- `title`: The title of the post
- `url`: The URL that the posts links to, if it the post has a URL
- `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: The number of comments that were made on the post
- `author`: The username of the person who submitted the post
- `created_at`: The date and time at which the post was submitted

# Extracting Ask and Show HN posts

Now we will further clean the data to categorize posts into those that contain Ask HN, Show HN or other.  We will do this by creating three lists and extracting the posts to their corresponding category.  One thing we have to account for is that not all posts have formatted capitalization.  Therefore, we will need to use the `string.lower()` type to make sure capitalization is not an issue in categorizing the posts.


In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Length of Show HN: ",len(ask_posts))
print("Length of Ask HN: ",len(show_posts))
print("Length of All Others: ",len(other_posts))

Length of Show HN:  1744
Length of Ask HN:  1162
Length of All Others:  17194


Now that we have the posts issued into lists based on their category, let's analyze the comments section.  We will be looking at how many times an individual post has been commented on by using the `num_comments` field.

In [3]:
#Ask_HN comments averaged
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
avg_ask = total_ask_comments / len(ask_posts)

print("Total comments for Ask HN: {:,.2f}".format(avg_ask))

Total comments for Ask HN: 14.04


In [4]:
#Show_HN comments averaged
total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
avg_show = total_show_comments / len(show_posts)
print("Total comments for Show HN: {:,.2f}".format(avg_show))

Total comments for Show HN: 10.32


We can see that Ask HN posts receive 50% more comments on average than Show HN comments.  This is probably due to the fact that Ask HN posts are asking the community for opinion or help.  And the Hacker News community is active in answering.  Though the community may not be as active in commenting on new products or projects.  

Since Ask HN posts receive more posts, we will concentrate on those records for our next analysis.

#  Determing what time most posts are commented on.

For our next analysis, we want to look at when most posts are commented on, rounded to the nearest hour.  This will help us determine when a post is more likely to be commented on, increasing our chance that an Ask HN post that we create will get more answers.  We will do this by:
1. Find the amount of ask posts created during each hour of the day, along with the number of comments those posts received.
2. Calculate the average amount of comments ask posts created at each hour of the day receive.

We will do that by creating two dictionaries:
- `counts_by_hour`: the number of **posts** created during each hour of the day.
- `comments_by_hour`: the number of **comments** generated at each hour.

In [57]:
print(ask_posts[:1])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']]


In [60]:
import datetime as dt

result_list = []
for row in ask_posts:
    result_list.append([row[6],int(row[4])])
    
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for post in result_list:
    date = post[0]
    no_comm = post[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = no_comm
    else:
        counts_by_hour[time] += 1
        comments_by_hour[time] += no_comm  

comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

Now we will calculate the average number of comments for posts created per each hour of the day as a new list called `avg_by_hour`.

In [42]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr,comments_by_hour[hr]/counts_by_hour[hr]])

avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

We will format the list to more readable and display the 5 highest values.

In [52]:
swap_avg_by_hour = []

for item in avg_by_hour:
    swap_avg_by_hour.append([item[1],item[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

sorted_swap

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [61]:
#Sort the values and print the 5 hours with the highest average comments:
print("Top 5 Hours for Asks Posts Comments:")

for avg,hr in sorted_swap[:5]:
    print("{}: {:.2f} average comments per post".format(dt.datetime.strptime(hr,"%H").strftime("%H:%M"),avg))

Top 5 Hours for Asks Posts Comments:
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


We can see that the most average comments of 38.59 were done at 3pm Eastern Standard Time (the time zone indicated in the original [dataset](https://www.kaggle.com/hacker-news/hacker-news-posts/home).  There is about a 60% increase between the highest and second highest average number of comments.

# Comparing upvotes between Show and Ask HN

Next we will add an analysis point to our Show HN and Ask HN datasets.  We will compare the upvotes vs downvotes for votes.  

# Upvotes per hour

Now we will determine if there are certain hours of the day that a post is given more points.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as an **Ask Post** and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, on average:
1. **Ask Posts** received more comments AND
2. **Ask Posts** created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments.